In [1]:
import os, sys
os.environ['ON_PY']="1"
from params import parser
from common_utils import setup_seed
from data_utils import pack_data_from_config
import numpy as np
from test_script.base import Test
import matplotlib.pyplot as plt
import numpy as np

notebook_dir = os.getcwd()
print(notebook_dir)
# 将工作目录更改为上一级目录
os.chdir("/home/wlc/FJSP-DRL-main")


pygame 2.6.0 (SDL 2.28.4, Python 3.10.14)
Hello from the pygame community. https://www.pygame.org/contribute.html
/home/wlc/FJSP-DRL-main/test_script


In [2]:
instances = [ ]
test_data_list = [ ]

args = ["--test_data", "14x11+mix",
        "--test_model", "14x11",]

print(args)


['--test_data', '14x11+mix', '--test_model', '14x11']


In [12]:
# DAN 解
configs = parser.parse_args(args=args)

setup_seed(configs.seed_test)

test_model = []

for model_name in configs.test_model:
    test_model.append((f'./trained_network/{configs.model_source}/{model_name}.pth', model_name))
print(test_model)
test_data = pack_data_from_config(configs.data_source, configs.test_data)
# print(test_data)
makespans = []
for i in range(len(test_model)):
    model = test_model[i]
    data = test_data[i][0]
    print(data[1][0])
    result_5_times = []
    for j in range(2):
        test = Test(configs, data, model[0])     
        result = test.greedy_strategy()
        result_5_times.append(result)
        test.env.render()
        tasks_data = test.env.tasks_data
    result_5_times = np.array(result_5_times)

    save_result = np.min(result_5_times, axis=0)
    print("testing results:")
    print(f"makespan(greedy): ", save_result[:, 0].mean())
    makespans.append(save_result[:, 0].mean())
    print("="*100)
print(tasks_data)
baseline_makespans = np.array(makespans)
print(baseline_makespans)



[('./trained_network/SD1/14x11.pth', '14x11')]
[[ 0  0 11 ...  0 10 11]
 [ 3  0  0 ...  3  3  0]
 [ 2  2  2 ...  2  2  2]
 ...
 [ 0  0  0 ...  1  1  1]
 [14 11 15 ... 13 15 16]
 [ 0  6  0 ...  7  6  0]]


testing results:
makespan(greedy):  165.8
[{'Task': 'Job', 'Station': 'Machine10', 'Start': 0, 'Duration': 0, 'Width': 0.4}, {'Task': 'Job', 'Station': 'Machine9', 'Start': 0, 'Duration': 0, 'Width': 0.4}, {'Task': 'Job', 'Station': 'Machine8', 'Start': 0, 'Duration': 0, 'Width': 0.4}, {'Task': 'Job', 'Station': 'Machine7', 'Start': 0, 'Duration': 0, 'Width': 0.4}, {'Task': 'Job', 'Station': 'Machine6', 'Start': 0, 'Duration': 0, 'Width': 0.4}, {'Task': 'Job', 'Station': 'Machine5', 'Start': 0, 'Duration': 0, 'Width': 0.4}, {'Task': 'Job', 'Station': 'Machine4', 'Start': 0, 'Duration': 0, 'Width': 0.4}, {'Task': 'Job', 'Station': 'Machine3', 'Start': 0, 'Duration': 0, 'Width': 0.4}, {'Task': 'Job', 'Station': 'Machine2', 'Start': 0, 'Duration': 0, 'Width': 0.4}, {'Task': 'Job', 'Station': 'Machine1', 'Start': 0, 'Duration': 0, 'Width': 0.4}, {'Task': 'Job', 'Station': 'Machine0', 'Start': 0, 'Duration': 0, 'Width': 0.4}, {'Task': 'Job2', 'Station': 'Machine3', 'Start': 0.0, 'Duration':

In [ ]:
import plotly.graph_objects as go
from plotly.colors import qualitative
import colorsys
import random

def render(tasks_data):
    '''tasks_data = [
    {"Task": "Job1-Task1", "Station": "Machine1", "Start": 0, "Duration": 4, "Width": 0.4},
    {"Task": "Job2-Task1", "Station": "Machine2", "Start": 5, "Duration": 3, "Width": 0.4},
    {"Task": "Job3-Task1", "Station": "Machine3", "Start": 9, "Duration": 2, "Width": 0.4},
    ]   '''
    fig = go.Figure()
    # 获取唯一的Job名称列表
    unique_jobs = list(set(task['Task'] for task in tasks_data))

    # 使用Plotly的定性颜色循环作为基础
    color_sequence = qualitative.Plotly

    # 如果Job数量超过内置颜色，动态生成更多颜色
    if len(unique_jobs) > len(color_sequence):
        extra_colors_needed = len(unique_jobs) - len(color_sequence)
        # 动态生成新颜色
        random.seed(42)  # 固定种子以保证结果可复现
        extra_colors = []
        for _ in range(extra_colors_needed):
            h, s, l = random.random(), 0.5 + random.random() / 2.0, 0.4 + random.random() / 5.0
            r, g, b = colorsys.hls_to_rgb(h, l, s)
            extra_colors.append(f"rgb({int(r*255)}, {int(g*255)}, {int(b*255)})")
        color_sequence = color_sequence + extra_colors

    # 创建颜色映射
    color_map = {job: color for job, color in zip(unique_jobs, color_sequence)}

    # 清空现有的绘图数据
    fig.data = []

    # 绘制条形图
    for task in tasks_data:
        fig.add_trace(go.Bar(
            x=[task["Duration"]],
            y=[task["Station"]],
            base=[task["Start"]],
            width=[task["Width"]],
            orientation='h',
            name=task["Task"],
            marker_color=color_map[task["Task"]],
        ))

    # 更新图表布局
    fig.update_layout(
        title="按Job上色的FJSP调度示意图 - 多种颜色",
        xaxis_title="时间",
        yaxis=dict(
            type='category',
            categoryorder='array',
            categoryarray=[task['Station'] for task in tasks_data]
        ),
        barmode='stack',
        legend_title="Job",
    )

    # 显示图表
    fig.show()
tasks_data_break5 = [
    task for task in tasks_data
    if not (task["Start"] + task["Duration"] > 40 and task["Station"] == "Machine5")
]


render(tasks_data)
render(tasks_data_break5)